# TP Recommendation
Loïc Herbelot

### Question 1.1 : Chargement des données

In [2]:
import numpy as np
from scipy import sparse


def load_movielens(filename, minidata=False):
    """
    Cette fonction lit le fichier filename de la base de donnees
    Movielens, par exemple 
    filename = '~/datasets/ml-100k/u.data'
    Elle retourne 
    R : une matrice utilisateur-item contenant les scores
    mask : une matrice valant 1 si il y a un score et 0 sinon
    """

    data = np.loadtxt(filename, dtype=int)

    R = sparse.coo_matrix((data[:, 2], (data[:, 0]-1, data[:, 1]-1)),
                          dtype=float)
    R = R.toarray()  # not optimized for big data

    # code la fonction 1_K
    mask = sparse.coo_matrix((np.ones(data[:, 2].shape),
                              (data[:, 0]-1, data[:, 1]-1)), dtype=bool )
    mask = mask.toarray()  # not optimized for big data

    if minidata is True:
        R = R[0:100, 0:200].copy()
        mask = mask[0:100, 0:200].copy()

    return R, mask


def objective(P, Q0, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q0.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))

    grad_P = -Q0.T.dot(tmp) + rho*P

    return val, grad_P


def total_objective(P, Q, R, mask, rho):
    """
    La fonction objectif du probleme complet.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q : la variable matricielle de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    grad_Q : le gradient par rapport a Q
    """

    tmp = (R - Q.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P ** 2))

    grad_P = 0  # todo

    grad_Q = 0  # todo

    return val, grad_P, grad_Q


def total_objective_vectorized(PQvec, R, mask, rho):
    """
    Vectorisation de la fonction precedente de maniere a ne pas
    recoder la fonction gradient
    """

    # reconstruction de P et Q
    n_items = R.shape[1]
    n_users = R.shape[0]
    F = PQvec.shape[0] / (n_items + n_users)
    Pvec = PQvec[0:n_items*F]
    Qvec = PQvec[n_items*F:]
    P = np.reshape(Pvec, (F, n_items))
    Q = np.reshape(Qvec, (n_users, F))

    val, grad_P, grad_Q = total_objective(P, Q, R, mask, rho)
    return val, np.concatenate([grad_P.ravel(), grad_Q.ravel()])


In [3]:
import numpy as np

filename = "ml-100k/u.data"

#sans "minidata"
R, mask = load_movielens(filename)
print("Taille de R:", R.shape)
print("Taille de mask:", mask.shape)

Taille de R: (943, 1682)
Taille de mask: (943, 1682)


In [4]:
#avec "minidata"
R_mini, mask_mini = load_movielens(filename, minidata=True)
print("Taille de R_mini:", R_mini.shape)
print("Taille de mask_mini:", mask_mini.shape)

Taille de R_mini: (100, 200)
Taille de mask_mini: (100, 200)


### Fonction de `minidata` :
Comme on peut le voir dans le code source, l'argument `minidata` sert à indiquer si l'on veut une plus petite partie des informations, seulement 100 lignes et 200 colonnes.

### Question 1.2 : Informations sur les données

In [5]:
with open("ml-100k/u.info") as f:
    print("Contenu du fichier d'info : \n"+ f.read())

Contenu du fichier d'info : 
943 users
1682 items
100000 ratings



On voit qu'on a donc :
 * 943 utilisateurs
 * 1682 films
 * Pour un total de 100 000 notes.

### Question 1.3 : Étude de la fonction objectif

Ici j'ai tracé la fonction avec deux variables :
![Graphe de la fonction objectif avec deux variables](plot_objective_2_var.png)


On voit donc que la fonction n'est pas convexe.

Le gradient de f est :

$\nabla f(P,Q) = (Q^T (1_K ⋅ (QP-R)) + \rho P, 1_K ⋅ (QP-R)P^T + \rho Q)$

Le gradient de $f$ n'est pas lipschitzien, car ses dérivées partielles ne sont pas bornées.

Exemple en dimension 1 :
$f(x,y) = \frac 1 2 (r- xy)^2 + \frac \rho 2 (x^2 + y^2) $

$\frac{df}{dx}(x) = x(y^2 + \rho) - yr$ et n'est pas bornée

### Question 2.1 :

La fonction $g$ est convexe.

Son gradient vaut : $\nabla g(P) = -{Q^0}^T(1_K ⋅ (R - Q^0*P)) + \rho P $



### Question 2.2 : Calcul du gradient de $g$

In [6]:
from scipy.optimize import check_grad

# Initialisation des variables :
rho = 0.2
P = np.random.random((7, 1682))*100

# Fonctions utilisées pour la vérification du gradient :

def g(P):
    """ Renvoie la sortie de la fonction objectif.
    :param P: un vecteur ligne de 7*1682 éléments
    :return : la sortie de la fonction objectif."""
    P = P.reshape((7, 1682))
    return objective(P, Q0, R, mask, rho)[0]

def grad_g(P):
    """Renvoie le gradient de la fonction objectif au point P.
    :param P: un vecteur ligne de 7*1682 éléments
    :return: le gradient sous forme de vecteur ligne."""
    P = P.reshape((7, 1682))
    return objective(P, Q0, R, mask, rho)[1].ravel()

# Activer/Désactiver la vérification du gradient :
verif = False # Choix de l'utilisateur
if verif :
    # Attention, calcul très long :
    print("Erreur sur le gradient :")
    for i in range(1):
        Q0 = np.random.random((943, 7))
        P = np.random.random((7, 1682))
        print("Vérification n°%d :" % i)
        print(check_grad(lambda P:objective(P.reshape((7, 1682)), Q0, R, mask, rho)[0],
                         lambda P:objective(P.reshape((7, 1682)), Q0, R, mask, rho)[1].ravel(),
                         P.ravel()))
else :
    print("Vérification sautée.")


Vérification sautée.


### Question 2.3 : Gradient à pas constant

In [7]:
from scipy.sparse.linalg import svds

def gradient(g, P0, gamma, epsilon, n_iter=1000):
    """ Minimise une fonction objectif par la méthode du gradient à pas constant.
    :param g:       La fonction à minimiser
    :param P0:      Le point de départ
    :param gamma:   Le pas
    :param epsilon: Critère d'arrêt : norme du gradient inférieure à epsilon
    :param n_iter:  Le nombre maximum d'itérations
    
    :return: Le point qui minimise la fonction, la valeur de la fonction en ce point, et son gradient
    """
    #Q0 est la matrice des 7 vecteurs singuliers à gauche de R
    # (|C| = 7)
    rho = 0.2
    Q0, s, vt = svds(R, min(7, min(R.shape)))
    x = P0
    grad = objective(x, Q0, R, mask, rho)[1]
    i = 0
#     L = rho + np.linalg.norm(Q0.T.dot(Q0), 'fro')
#     gamma = 1/L
    while i < n_iter and np.linalg.norm(grad, 'fro') > epsilon:
        val, grad = objective(x, Q0, R, mask, rho)
        if i%10 == 0:
            print("Iteration n°%5d, g(x) = %E" % (i, val))
        x -= gamma * grad
        i += 1
    print("Valeur maximale du gradient en ce point : ",np.max(grad))
    return x, val, grad

### Question 2.4 : Démonstration de la fonction `gradient` :

In [8]:
# Trouvons le P qui minimise g :
x, val, grad = gradient(g, np.random.random((7, 1682))*1000, 1, 0.1, n_iter=1000)
print("Minimum trouvé : %E" % val)

# Essayons 10 valeurs au hasard pour vérifier la pertinence du résultat :
print("10 valeurs au hasard :")
rho = 0.2
Q0, s, vt = svds(R, min(7, min(R.shape)))
for i in range(10):
    # J'évite de prendre des matrices "proches" de 0, 
    # sinon on ne pourrait pas voir comment se comporte g.
    # Je multiplie une matrice "random" par 10^[un entier au hasard]
    exp = np.random.randint(-1, 5)
    P = np.random.random((7, 1682))*10**exp
    rand_val = objective(P, Q0, R, mask, rho)[0]
    if rand_val < val:
        print("Nouveau minimum trouvé :")
    print("g(P) = %E" % (rand_val))

Iteration n°    0, g(x) = 5.780278E+08
Iteration n°   10, g(x) = 2.523856E+06
Iteration n°   20, g(x) = 3.173443E+05
Iteration n°   30, g(x) = 2.980205E+05
Iteration n°   40, g(x) = 2.978292E+05
Iteration n°   50, g(x) = 2.978272E+05
Valeur maximale du gradient en ce point :  0.00336705411742
Minimum trouvé : 2.978272E+05
10 valeurs au hasard :
g(P) = 6.783189E+05
g(P) = 5.961358E+06
g(P) = 6.261511E+10
g(P) = 6.236840E+10
g(P) = 6.044102E+10
g(P) = 6.215197E+10
g(P) = 5.998706E+06
g(P) = 6.854912E+05
g(P) = 6.636624E+05
g(P) = 6.614186E+05


### Question 3.1 : Recherche linéaire :

In [9]:
from scipy.sparse.linalg import svds

def lin_gradient(g, P0, epsilon, n_iter=1000):
    """ Minimise une fonction objectif par la méthode du gradient à pas constant.
    :param g:       La fonction à minimiser
    :param P0:      Le point de départ
    :param epsilon: Critère d'arrêt : norme du gradient inférieure à epsilon
    :param n_iter:  Le nombre maximum d'itérations
    
    :return: Le point qui minimise la fonction, la valeur de la fonction en ce point, et son gradient
    """
    #Q0 est la matrice des 7 vecteurs singuliers à gauche de R
    # (|C| = 7)
    rho = 0.2
    Q0, s, vt = svds(R, min(7, min(R.shape)))
    x = P0
    grad = objective(x, Q0, R, mask, rho)[1]
    i = 0
    gamma_values = np.linspace(0.1, 10, 100)
#     L = rho + np.linalg.norm(Q0.T.dot(Q0), 'fro')
#     gamma = 1/L
    while i < n_iter and np.linalg.norm(grad, 'fro') > epsilon:
        g = lambda x:objective(x, Q0, R, mask, rho)
        val, grad = g(x)
        # Affichage des résultats :
        if i%10 == 0:
            print("Iteration n°%5d, g(x) = %E" % (i, val))
        id_argmin = np.argmin(lambda gamma: g(x - gamma*grad))
        x -= gamma_values[id_argmin] * grad
        i += 1
    print("Valeur maximale du gradient en ce point : ",np.max(grad))
    return x, val, grad


# Trouvons le P qui minimise g :
x, val, grad = lin_gradient(g, np.random.random((7, 1682))*1000, 0.1, n_iter=1000)
print("Minimum trouvé : %E" % val)

# Essayons 10 valeurs au hasard pour vérifier la pertinence du résultat :
print("10 valeurs au hasard :")
rho = 0.2
Q0, s, vt = svds(R, min(7, min(R.shape)))
for i in range(10):
    # J'évite de prendre des matrices "proches" de 0, 
    # sinon on ne pourrait pas voir comment se comporte g.
    # Je multiplie une matrice "random" par 10^[un entier au hasard]
    exp = np.random.randint(-1, 5)
    P = np.random.random((7, 1682))*10**exp
    rand_val = objective(P, Q0, R, mask, rho)[0]
    if rand_val < val:
        print("Nouveau minimum trouvé :")
    print("g(P) = %E" % (rand_val))

Iteration n°    0, g(x) = 6.009949E+08
Iteration n°   10, g(x) = 2.952142E+08
Iteration n°   20, g(x) = 1.613289E+08
Iteration n°   30, g(x) = 9.352069E+07
Iteration n°   40, g(x) = 5.615438E+07
Iteration n°   50, g(x) = 3.450660E+07
Iteration n°   60, g(x) = 2.156385E+07
Iteration n°   70, g(x) = 1.366078E+07
Iteration n°   80, g(x) = 8.762728E+06
Iteration n°   90, g(x) = 5.693684E+06
Iteration n°  100, g(x) = 3.754568E+06
Iteration n°  110, g(x) = 2.521342E+06
Iteration n°  120, g(x) = 1.732921E+06
Iteration n°  130, g(x) = 1.226702E+06
Iteration n°  140, g(x) = 9.005132E+05
Iteration n°  150, g(x) = 6.896940E+05
Iteration n°  160, g(x) = 5.530874E+05
Iteration n°  170, g(x) = 4.643715E+05
Iteration n°  180, g(x) = 4.066448E+05
Iteration n°  190, g(x) = 3.690182E+05
Iteration n°  200, g(x) = 3.444556E+05
Iteration n°  210, g(x) = 3.283994E+05
Iteration n°  220, g(x) = 3.178910E+05
Iteration n°  230, g(x) = 3.110059E+05
Iteration n°  240, g(x) = 3.064904E+05
Iteration n°  250, g(x) =